In [ ]:
import sys
import os
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

# Add the current directory to the path to import our database config
sys.path.append(os.path.dirname(__file__))

from database_config import get_db_connection
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)

def query_database():
    """
    Query the database and display information about the stored data
    Uses the new database configuration system (PostgreSQL or SQLite)
    """
    try:
        # Get database connection
        db_conn = get_db_connection()
        
        # Test connection
        if not db_conn.test_connection():
            print("Error: Cannot connect to database. Check your configuration.")
            return
        
        # Check if table exists
        if not db_conn.table_exists():
            print("Error: option_strategies table does not exist.")
            return
        
        # Get record count
        count_result = db_conn.execute_query("SELECT COUNT(*) FROM option_strategies")
        total_records = count_result[0][0] if count_result else 0
        
        print(f"Database type: {db_conn.config.db_type.upper()}")
        print(f"Total records: {total_records}")
        
        if total_records == 0:
            print("No records found in the database.")
            return
        
        # Get count by strategy type
        print("\nRecords by strategy type:")
        strategy_results = db_conn.execute_query(
            "SELECT strategy_type, COUNT(*) as count FROM option_strategies GROUP BY strategy_type"
        )
        for strategy, count in strategy_results:
            print(f"  {strategy}: {count} records")
        
        # Get count by tab
        print("\nRecords by strategy and tab:")
        tab_results = db_conn.execute_query(
            "SELECT strategy_type, tab_name, COUNT(*) as count FROM option_strategies GROUP BY strategy_type, tab_name"
        )
        for strategy, tab, count in tab_results:
            print(f"  {strategy} - {tab}: {count} records")
        
        # Get most recent scrape date
        recent_result = db_conn.execute_query("SELECT MAX(scrape_date) FROM option_strategies")
        most_recent = recent_result[0][0] if recent_result else None
        print(f"\nMost recent scrape: {most_recent}")
        
        # Get ticker distribution
        print("\nTop 10 tickers:")
        ticker_results = db_conn.execute_query("""
            SELECT ticker, COUNT(*) as count 
            FROM option_strategies 
            WHERE ticker != 'N/A' 
            GROUP BY ticker 
            ORDER BY count DESC 
            LIMIT 10
        """)
        for ticker, count in ticker_results:
            print(f"  {ticker}: {count} records")
        
        # Display sample records using DataFrame
        print("\nSample records (most recent 5):")
        df = db_conn.execute_query_df("""
            SELECT strategy_type, tab_name, ticker, trigger_price, strike_price, estimated_premium, options_expiry_date
            FROM option_strategies
            ORDER BY scrape_date DESC
            LIMIT 5
        """)
        
        print(df)
        
    except Exception as e:
        print(f"Error querying database: {str(e)}")

def export_to_csv(output_file=None):
    """
    Export the database to a CSV file
    Uses the new database configuration system
    
    Parameters:
    output_file (str): Path for the output CSV file. If None, a timestamped filename is used.
    
    Returns:
    str: Path to the created CSV file
    """
    try:
        # Get database connection
        db_conn = get_db_connection()
        
        # Test connection
        if not db_conn.test_connection():
            print("Error: Cannot connect to database. Check your configuration.")
            return None
        
        # Check if table exists
        if not db_conn.table_exists():
            print("Error: option_strategies table does not exist.")
            return None
        
        # Create output filename if not provided
        if output_file is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_file = f"option_strategies_export_{timestamp}.csv"
        
        # Query all data
        df = db_conn.execute_query_df("SELECT * FROM option_strategies")
        
        # Export to CSV
        df.to_csv(output_file, index=False)
        
        print(f"Successfully exported {len(df)} records to {output_file}")
        return output_file
        
    except Exception as e:
        print(f"Error exporting database: {str(e)}")
        return None

if __name__ == "__main__":
    # Query the database
    query_database()
    
    # Uncomment to export to CSV
    # export_to_csv()

In [ ]:
# Test the updated database query functions
query_database()

In [ ]:
def get_strategies_by_date(scrape_date=None):
    """
    Query option strategies from the database for a specific scrape date
    Uses the new database configuration system
    
    Parameters:
    scrape_date (str): Specific scrape date to query for (ISO format, e.g. '2025-03-08T14:30:45.123456')
                       If None, returns most recent scrape date's data
    
    Returns:
    pandas.DataFrame: DataFrame containing the matched records
    """
    try:
        # Get database connection
        db_conn = get_db_connection()
        
        # Test connection
        if not db_conn.test_connection():
            print("Error: Cannot connect to database. Check your configuration.")
            return pd.DataFrame()
        
        # If no specific date provided, get the most recent date
        if scrape_date is None:
            query = "SELECT MAX(scrape_date) FROM option_strategies"
            result = db_conn.execute_query(query)
            most_recent = result[0][0] if result else None
            scrape_date = most_recent
            print(f"Using most recent scrape date: {scrape_date}")
        
        # Query using date prefix (to handle time portion variations)
        date_prefix = scrape_date.split('T')[0] if 'T' in scrape_date else scrape_date
        
        # Use LIKE for both PostgreSQL and SQLite compatibility
        query = """
            SELECT * FROM option_strategies 
            WHERE scrape_date::text LIKE %s
            ORDER BY strategy_type, tab_name
        """ if db_conn.config.is_postgresql() else """
            SELECT * FROM option_strategies 
            WHERE scrape_date LIKE ?
            ORDER BY strategy_type, tab_name
        """
        
        # Execute query and load into DataFrame
        df = db_conn.execute_query_df(query, (f"{date_prefix}%",))
        
        # Display summary information
        if len(df) > 0:
            print(f"Found {len(df)} records from {date_prefix}")
            print(f"Strategies: {', '.join(df['strategy_type'].unique())}")
        else:
            print(f"No records found for date {date_prefix}")
        
        return df
        
    except Exception as e:
        print(f"Error querying database: {str(e)}")
        return pd.DataFrame()

# Test the function
specific_date = "2025-03-08"
specific_strategies = get_strategies_by_date(specific_date)
specific_strategies